In [1]:
from pyspark.sql.functions import *

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn import metrics
import numpy as np
# from sklearn.externals.six import StringIO 
from six import StringIO
from IPython.display import Image  
import pydotplus

import os

In [8]:
file_cache = 'data/elo7_search_dataset.csv'
if os.path.exists(file_cache):
    df = pd.read_csv(file_cache)
    print("Using cache")
else:
    df = pd.read_csv('s3://elo7-datasets/elo7-search-dataset/elo7_search_dataset.csv')
    print("File downloaded")
    df.to_csv(file_cache)
    print("Cache created")

Using cache


In [12]:
df['price'] = np.round(df['price'],2)
df.rename(columns={"classification": "product_relevance_classification"})

,Unnamed: 0,product_id,seller_id,query,search_page,position,product_relevance_classification,title,category,subcategory,tags,collection_names,creation_date,price,minimum_quantity,weight,volume,express_delivery,order_counts,view_counts
0,0,14229551,5243703,embalagem papel kraft,3,33,0,Sacola kraft media c/ papel rendado,Papel e Cia,Embalagem,"sacolas de papel,embalagens para presente,bols...",sacolas papel coloridas niver anna's,2008-11-05 22:57:17,3.39,4,6.0,NaN,0,8.0,131.0
1,1,14229551,5243703,papel rendado,2,14,0,Sacola kraft media c/ papel rendado,Papel e Cia,Embalagem,"sacolas de papel,embalagens para presente,bols...",sacolas papel coloridas niver anna's,2008-11-05 22:57:17,12.04,0,6.0,NaN,0,9.0,133.0
2,2,14229551,5243703,sacola de papel kraft,1,21,1,Sacola kraft media c/ papel rendado,Papel e Cia,Embalagem,"sacolas de papel,embalagens para presente,bols...",sacolas papel coloridas niver anna's,2008-11-05 22:57:17,12.26,4,6.0,NaN,0,7.0,129.0
3,3,4564841,9070334,tag para bijuteria,1,20,0,Papelaria impressa para bijuteria 009,Papel e Cia,Tag,"tags,tag,bijuterias,brinco,pulseira,colar,etiq...",kit bijuterias,2009-10-27 03:35:16,89.40,4,6.0,NaN,1,NaN,111.0
4,4,4564841,9070334,bijuterias,1,31,0,Papelaria impressa para bijuteria 009,Papel e Cia,Tag,"tags,tag,bijuterias,brinco,pulseira,colar,etiq...",kit bijuterias,2009-10-27 03:35:16,89.66,1,8.0,NaN,1,NaN,127.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509910,509910,5118671,799465,lembrancinha dia dos pais em goiania,1,21,1,Copo Long Drink Dia dos Pais,Lembrancinhas,Personalizadas,"copo long drink,copo para formatura,copos de a...",copo long drink dia pais,2019-07-31 17:14:05,1.99,57,7.0,NaN,1,NaN,32.0
509911,509911,10373066,3222466,mesa infantil com 2 cadeiras,1,1,0,Mesinha Infantil com 2 Cadeiras e Tampo com 55...,Decoração,Móveis,"mesa infantil,mesa,mesa de mdf,mesa para crian...",mesas banquetas cadeiras infantis,2019-08-01 16:56:22,359.00,1,9500.0,136146.0,0,17.0,7.0
509912,509912,1473221,1039041,vela 40 anos,1,37,0,Vela personalizada 40 anos em biscuit,Aniversário e Festas,Topo de Bolo,"vela,vela personalizada,vela de biscuit,vela d...",velas topo bolo,2019-08-03 02:02:02,70.69,1,127.0,896.0,1,NaN,NaN
509913,509913,10389379,510188,pai ciclista,1,13,1,Kit Tal Pai Tal Filho Ciclista,Roupas,Camiseta,"bebê,papai,mamãe,mãe,pai,passear,personalizado...",pais,2019-08-03 15:34:48,65.25,8,8.0,NaN,1,NaN,6.0


- `product_id`: product number identification
- `seller_id`: seller number identification
- `query`: the text inserted by users
- `search_page`: the page number the product appeared (min 1 and max 5)
- `position`: the position the product appeared in the search page (min 0 and max 38)
- `product_relevance_classification`: a measure of relevance. 1 means the product is relevant for the given query and 0 it is not
- `title`: product title
- `category`: product category
- `subcategory`: product subcategory
- `tags`: product tags inserted by the seller
- `collection_names`: a list of collection names (created by the seller) for which the product is in
- `creation_date`: the date of product registration in Elo7 platform
- `price`: the product price (R$)
- `minimum_quantity`: the minimum quantity the seller sells the product
- `weight`: the weight (grams) of a product unit
- `volume`: the volume (cubic meters) of a product unit
- `express_delivery`: indicates if the product has already been made (1) or not (0)
- `order_counts`: the number of times the product was purchased in the last three months
- `view_counts`: the number of times the product was clicked in the last three months

In [14]:
numeric_features = ["search_page", "position", "price", "minimum_quantity", "weight", "order_counts", "view_counts"]
categoric_features = ['category']